In [1]:
import pandas as pd
import numpy as np
import subprocess

bed_prefix = "/LARGE0/gr10478/b37974/Pulmonary_Hypertension/cteph_agp3k/wgs/18.clean_gt/cteph_agp3k.clean"
sep_prefix = "maf_check"

aaf_output = f"{sep_prefix}.aaf"
# subprocess.run([
#     "/home/b/b37974/plink2",
#     "--bfile", bed_prefix,
#     "--freq",
#     "--out", aaf_output, 
#     "--threads", "8"
# ])

In [2]:
columns_afreq = ['ID', 'ALT_FREQS']
columns_hardy = ['ID', 'P']
chunksize = 100000  # 可适当加大以减少IO

# 读取 afreq 文件并计算 MAF
afreq_chunks = []
for chunk in pd.read_csv(f"{aaf_output}.afreq", delim_whitespace=True,
                         usecols=columns_afreq, chunksize=chunksize,
                         dtype={'ID': str, 'ALT_FREQS': float}):
    chunk['MAF'] = chunk['ALT_FREQS'].clip(upper=1).apply(lambda x: min(x, 1 - x))
    afreq_chunks.append(chunk[['ID', 'MAF']])
afreq_df = pd.concat(afreq_chunks, ignore_index=True)


/tmp/ipykernel_1494654/884979755.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  for chunk in pd.read_csv(f"{aaf_output}.afreq", delim_whitespace=True,


In [3]:
mono_variants = afreq_df[afreq_df['MAF'] == 0].copy()
mono_variants["ID"].to_csv("case_control.mono_variants", index=False, header=False)

In [4]:
rm_mono_prefix = "cteph_agp3k.clean.rm_mono"

# subprocess.run([
#     "/home/b/b37974/plink2",
#     "--bfile", bed_prefix,
#     "--exclude", "case_control.mono_variants",
#     "--make-bed",
#     "--out", rm_mono_prefix, 
#     "--threads", "8"
# ])

In [5]:
case_aaf_output = "case.aaf"
control_aaf_output = "control.aaf"

# subprocess.run([
#     "/home/b/b37974/plink2",
#     "--bfile", rm_mono_prefix,
#     "--freq",
#     "--keep-if", "PHENO1==2", 
#     "--out", case_aaf_output,
#     "--threads", "8"
# ])

# subprocess.run([
#     "/home/b/b37974/plink2",
#     "--bfile", rm_mono_prefix,
#     "--freq",
#     "--keep-if", "PHENO1==1", 
#     "--out", control_aaf_output,
#     "--threads", "8"
# ])

In [6]:
columns_afreq = ['ID', 'ALT_FREQS']
columns_hardy = ['ID', 'P']
chunksize = 100000  # 可适当加大以减少IO

# 读取 afreq 文件并计算 MAF
afreq_chunks = []
for chunk in pd.read_csv(f"{control_aaf_output}.afreq", delim_whitespace=True,
                         usecols=columns_afreq, chunksize=chunksize,
                         dtype={'ID': str, 'ALT_FREQS': float}):
    chunk['MAF'] = chunk['ALT_FREQS'].clip(upper=1).apply(lambda x: min(x, 1 - x))
    afreq_chunks.append(chunk[['ID', 'MAF']])
control_afreq_df = pd.concat(afreq_chunks, ignore_index=True)

/tmp/ipykernel_1494654/3853250943.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  for chunk in pd.read_csv(f"{control_aaf_output}.afreq", delim_whitespace=True,


In [7]:
not_rare_variants = control_afreq_df[(control_afreq_df['MAF'] > 0.01) | (control_afreq_df['MAF'] == 0.01)].copy()
not_rare_variants["ID"].to_csv("control.not_rare_variants", index=False, header=False)

In [8]:
not_rare_prefix = "cteph_agp3k.clean.rm_mono.not_rare"

# subprocess.run([
#     "/home/b/b37974/plink2",
#     "--bfile", rm_mono_prefix,
#     "--extract", "control.not_rare_variants",
#     "--make-bed",
#     "--out", not_rare_prefix, 
#     "--threads", "8"
# ])

In [12]:
subprocess.run([
    "/home/b/b37974/plink2",
    "--bfile", not_rare_prefix,
    "--geno-counts",
    "--keep-if", "PHENO1==2", 
    "--out", "count_case",
    "--threads", "8"
])

subprocess.run([
    "/home/b/b37974/plink2",
    "--bfile", not_rare_prefix,
    "--geno-counts", 
    "--keep-if", "PHENO1==1", 
    "--out", "count_control",
    "--threads", "8"
])

PLINK v2.0.0-a.5.17LM 64-bit Intel (20 Oct 2024)   cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to count_case.log.
Options in effect:
  --bfile cteph_agp3k.clean.rm_mono.not_rare
  --geno-counts
  --keep-if PHENO1==2
  --out count_case
  --threads 8

Start time: Fri May 30 13:15:29 2025
515039 MiB RAM detected, ~401776 available; reserving 257519 MiB for main
workspace.
Using up to 8 compute threads.
3027 samples (1940 females, 1087 males; 3027 founders) loaded from
cteph_agp3k.clean.rm_mono.not_rare.fam.
6022542 variants loaded from cteph_agp3k.clean.rm_mono.not_rare.bim.
1 binary phenotype loaded (378 cases, 2649 controls).
--keep-if: 2649 samples removed.
378 samples (301 females, 77 males; 378 founders) remaining after main filters.
378 cases and 0 controls remaining after main filters.
Calculating allele frequencies... 1011131415161718192021222325262728293031323334353638394041424344454647485051525354555657585960

CompletedProcess(args=['/home/b/b37974/plink2', '--bfile', 'cteph_agp3k.clean.rm_mono.not_rare', '--geno-counts', '--keep-if', 'PHENO1==1', '--out', 'count_control', '--threads', '8'], returncode=0)